# Appendix 2: Using CAQL with Python

We make use of the Python Data Fetching API for illustration purposes.

* Code: github.com/circonus-labs/python-circonusapi
* Blog: https://www.circonus.com/2018/05/less-toil-more-coil-telemetry-analysis-with-python/

Using the API is not covered in this tutorial.
Everything we do here can be done with the UI as well.

**TODO:** Make python-circonusapi installable via pip including circonusdata

In [9]:
# Install python circonusapi
!cd /work/circ-workbench/src/python-circonusapi/ && python setup.py install

running install
running build
running build_py
running install_lib
running install_egg_info
Removing /opt/conda/lib/python3.6/site-packages/circonusapi-0.3.0-py3.6.egg-info
Writing /opt/conda/lib/python3.6/site-packages/circonusapi-0.3.0-py3.6.egg-info


In [27]:
%matplotlib inline
import json, os
from datetime import datetime

# We make use of Pandas for plotting
import pandas as pd

# Import the circonusapi wrapper
from circonusapi import circonusdata

# We store our API tokens in a JSON file with structure:
# { "<account>" : "<token>", ... }
with open(os.path.expanduser("/work/.circonusrc.json"),"r") as fh:
    config = json.load(fh)

## Fetch and plot a CAQL statement
# returns matpotlib axes object
def caql_plot(account, *args,**kwargs):
    # 1. Initiate a connection object
    circ = circonusdata.CirconusData(config[account])
    # 2. Make a CAQL query
    data=circ.caql(*args)
    # 3. Wrap as pandas dataframe
    df=pd.DataFrame(data)
    # 4. [Optional] Setup a "time" index-colum
    df['time']=pd.to_datetime(df['time'],unit='s')
    df.set_index('time', inplace=True)
    # 5. Plot!
    return df.plot(figsize=(20,5), legend=False, lw=.8, **kwargs)

In [28]:
from matplotlib import pyplot as plt

# Plot a circllhist as bar chart
def hist_plot(H,**kwargs):
    x=[] # midpoints
    h=[] # height
    w=[] # widths
    for b, c in H:
        x.append(b.midpoint())
        h.append(c / b.width())
        w.append(b.width())
    return plt.bar(x,h,w,**kwargs)

# Fetch and plot a Circonus histograms
def caql_hist_plot(account, *args, **kwargs):
    fig = plt.figure(figsize=(20,5))
    circ = circonusdata.CirconusData(config[account])
    data = circ.caql(*args)
    for H in data['output[0]']:
        hist_plot(H, alpha=0.2, **kwargs)
    return fig.get_axes()[0]